In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
from pyCRMtools.pycaps import arps_read
from pyCRMtools.pycaps import pycaps_fields
from pyCRMtools.pycaps import calvars_radar as pycaps_radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
%matplotlib inline

In [ ]:
from natsort import natsorted
radar_name = 'KHTX'
input_tag = 'filt_retr'
radar_type= 'NEXRAD'
radar_start_timestamp = '20160428060000'
radar_end_timestamp = '20160428060300'
el_req = 0.5
radar_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/2016/NEXRAD/IOP_4A_4C/HTX/CFRadial'
radar_paths = glob.glob(radar_dir + '/*20160428_060*{}*_{}.nc'.format(radar_name, input_tag))
radar_path_dict = radar.get_radar_paths(radar_paths, radar_start_timestamp, radar_end_timestamp,
                                        el_req=el_req, radar_type=radar_type)
print(radar_path_dict['sweeptimelist'])

In [ ]:
# radar_dict = radar.read_sweeps(radar_paths, starttime, stoptime)
radar_dict = radar.read_sweeps_new(radar_path_dict, el_req=el_req, radar_type=radar_type)
print(radar_dict['radarsweeplist'])
print(radar_dict['radarsweeplist'][0].info())

In [ ]:
import pyart
import cartopy.crs as ccrs

for sweeptime, radar_obj in zip(radar_dict['sweeptimelist'][:1], radar_dict['radarsweeplist'][:1]):
    
#     rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radar_obj, rhv_field='RHO', refl_field='REF',
#                                                               min_ncp=None, min_rhv=0.95,
#                                                               min_refl=5., max_refl=None) 
    #print(rhoHV_filter.gate_excluded)
#    radar_obj.fields['ZDR']['data'] = np.ma.masked_where(rhoHV_ref_filter.gate_excluded, radar_obj.fields['ZDR']['data'])
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    display = pyart.graph.RadarMapDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
#     projection = ccrs.LambertConformal(central_latitude=PIPS_lat,
#                                        central_longitude=PIPS_lon)
    # plot dBZ
    ax = fig.add_subplot(111)
    print("Time: ", sweeptime)
    print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('REF', 0, title='dBZ', cmap='pyart_HomeyerRainbow',
#                          vmin=0., vmax=60, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    # plot D0
#     ax = fig.add_subplot(111)
#     print("Time: ", radar_obj.metadata['start_datetime'])
#     print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('D0', 0, title='D0', cmap='viridis',
#                          vmin=0., vmax=4., colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
                         # min_lon=-88., max_lon=-86., min_lat=40., max_lat=41.)
    # plot ZDR
    
#     display.plot_ppi_map('ZDR_filtered', 0, title='ZDR', cmap='pyart_HomeyerRainbow',
#                          vmin=0, vmax=6, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    display.plot_ppi_map('RHO', 0, title='RHO', cmap='pyart_HomeyerRainbow',
                         vmin=0, vmax=1, colorbar_label='', ax=ax, resolution='10m', projection=projection,
                         fig=fig)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-50, 50), ylim=(-50, 50), ax=ax)
    #display.set_limits(xlim=(-75, -55), ylim=(75, 95), ax=ax)
    #display.plot_point(PIPS_lon, PIPS_lat, 'r*', ms=20)
    display.ax.set_extent([-100000, 100000, -100000, 100000], crs=projection)
    #display.ax.set_extent([40000, 90000, -30000, 20000], crs=projection)
    #ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'r*', ms=20)
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [ ]:
from scipy.signal import medfilt2d

for sweeptime, radar_obj in zip(radar_dict['sweeptimelist'][:1], radar_dict['radarsweeplist'][:1]):

    rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radar_obj, rhv_field='RHO', refl_field='REF',
                                                              min_ncp=None, min_rhv=0.95, min_refl=5.0, 
                                                              max_refl=None)
    
#     rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radar_obj, rhv_field='RHO', refl_field='REF',
#                                                               min_ncp=None, min_rhv=0.95,
#                                                               min_refl=5., max_refl=None) 
    #print(rhoHV_filter.gate_excluded)
#    radar_obj.fields['ZDR']['data'] = np.ma.masked_where(rhoHV_ref_filter.gate_excluded, radar_obj.fields['ZDR']['data'])
    
    radar_obj.add_field_like('REF', 'REF_filtered2',
                             radar_obj.fields['REF']['data'].copy(), replace_existing=True)

    radar_obj.fields['REF_filtered2']['data'] = \
        medfilt2d(radar_obj.fields['REF_filtered2']['data'], kernel_size=3)
    
    radar_obj.add_field_like('RHO', 'RHO_filtered2',
                             radar_obj.fields['RHO']['data'].copy(), replace_existing=True)

    radar_obj.fields['RHO_filtered2']['data'] = \
        medfilt2d(radar_obj.fields['RHO_filtered2']['data'], kernel_size=3)
    
    rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radar_obj, rhv_field='RHO_filtered2', 
                                                              refl_field='REF_filtered2',
                                                              min_ncp=None, min_rhv=0.95, min_refl=5.0, 
                                                              max_refl=None)
    
    radar_obj.fields['RHO_filtered2']['data'] = \
        np.ma.masked_where(rhoHV_ref_filter.gate_excluded,
                           radar_obj.fields['RHO_filtered2']['data'])
    
#     mask = radar_obj.fields['RHO_filtered2']['data'].mask

#     radar_obj.fields['RHO_filtered2']['data'] = \
#         medfilt2d(radar_obj.fields['RHO_filtered2']['data'], kernel_size=3)

#     radar_obj.fields['RHO_filtered2']['data'] = \
#         np.ma.masked_array(data=radar_obj.fields['RHO_filtered2']['data'],
#                            mask=mask)

    
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    display = pyart.graph.RadarMapDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
#     projection = ccrs.LambertConformal(central_latitude=PIPS_lat,
#                                        central_longitude=PIPS_lon)
    # plot dBZ
    ax = fig.add_subplot(111)
    print("Time: ", sweeptime)
    print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('REF', 0, title='dBZ', cmap='pyart_HomeyerRainbow',
#                          vmin=0., vmax=60, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    # plot D0
#     ax = fig.add_subplot(111)
#     print("Time: ", radar_obj.metadata['start_datetime'])
#     print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('D0', 0, title='D0', cmap='viridis',
#                          vmin=0., vmax=4., colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
                         # min_lon=-88., max_lon=-86., min_lat=40., max_lat=41.)
    # plot ZDR
    
#     display.plot_ppi_map('ZDR_filtered', 0, title='ZDR', cmap='pyart_HomeyerRainbow',
#                          vmin=0, vmax=6, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    display.plot_ppi_map('RHO_filtered2', 0, title='RHO', cmap='pyart_HomeyerRainbow',
                         vmin=0, vmax=1, colorbar_label='', ax=ax, resolution='10m', projection=projection,
                         fig=fig)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-50, 50), ylim=(-50, 50), ax=ax)
    #display.set_limits(xlim=(-75, -55), ylim=(75, 95), ax=ax)
    #display.plot_point(PIPS_lon, PIPS_lat, 'r*', ms=20)
    display.ax.set_extent([-100000, 100000, -100000, 100000], crs=projection)
    #display.ax.set_extent([40000, 90000, -30000, 20000], crs=projection)
    #ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'r*', ms=20)
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [ ]:
for sweeptime, radar_obj in zip(radar_dict['sweeptimelist'][:1], radar_dict['radarsweeplist'][:1]):
    
#     rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radar_obj, rhv_field='RHO', refl_field='REF',
#                                                               min_ncp=None, min_rhv=0.95,
#                                                               min_refl=5., max_refl=None) 
    #print(rhoHV_filter.gate_excluded)
#    radar_obj.fields['ZDR']['data'] = np.ma.masked_where(rhoHV_ref_filter.gate_excluded, radar_obj.fields['ZDR']['data'])
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    display = pyart.graph.RadarMapDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
#     projection = ccrs.LambertConformal(central_latitude=PIPS_lat,
#                                        central_longitude=PIPS_lon)
    # plot dBZ
    ax = fig.add_subplot(111)
    print("Time: ", sweeptime)
    print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('REF', 0, title='dBZ', cmap='pyart_HomeyerRainbow',
#                          vmin=0., vmax=60, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    # plot D0
#     ax = fig.add_subplot(111)
#     print("Time: ", radar_obj.metadata['start_datetime'])
#     print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('D0', 0, title='D0', cmap='viridis',
#                          vmin=0., vmax=4., colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
                         # min_lon=-88., max_lon=-86., min_lat=40., max_lat=41.)
    # plot ZDR
    
#     display.plot_ppi_map('ZDR_filtered', 0, title='ZDR', cmap='pyart_HomeyerRainbow',
#                          vmin=0, vmax=6, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    display.plot_ppi_map('RHO_filtered', 0, title='RHO', cmap='pyart_HomeyerRainbow',
                         vmin=0, vmax=1, colorbar_label='', ax=ax, resolution='10m', projection=projection,
                         fig=fig)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-50, 50), ylim=(-50, 50), ax=ax)
    #display.set_limits(xlim=(-75, -55), ylim=(75, 95), ax=ax)
    #display.plot_point(PIPS_lon, PIPS_lat, 'r*', ms=20)
    display.ax.set_extent([-100000, 100000, -100000, 100000], crs=projection)
    #display.ax.set_extent([40000, 90000, -30000, 20000], crs=projection)
    #ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'r*', ms=20)
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [ ]:
for radar_obj in radar_dict['radarsweeplist'][:1]:
    radar_obj.add_field_like('ZDR', 'ZDR_smooth', \
                             radar_obj.fields['ZDR']['data'].copy(), replace_existing=True)
    radar_obj.fields['ZDR_smooth']['data'] = medfilt2d(radar_obj.fields['ZDR_smooth']['data'])
    radar_obj.fields['ZDR_smooth']['data'] = np.ma.masked_array(data = radar_obj.fields['ZDR_smooth']['data'], 
                                                                mask = radar_obj.fields['ZDR']['data'].mask)
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    display = pyart.graph.RadarMapDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
#     projection = ccrs.LambertConformal(central_latitude=PIPS_lat,
#                                        central_longitude=PIPS_lon)
    # plot dBZ
    ax = fig.add_subplot(111)
    print("Time: ", radar_obj.metadata['start_datetime'])
    print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('REF', 0, title='dBZ', cmap='pyart_HomeyerRainbow',
#                          vmin=0., vmax=60, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    # plot D0
#     ax = fig.add_subplot(111)
#     print("Time: ", radar_obj.metadata['start_datetime'])
#     print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('D0', 0, title='D0', cmap='viridis',
#                          vmin=0., vmax=4., colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
                         # min_lon=-88., max_lon=-86., min_lat=40., max_lat=41.)
    # plot ZDR
    
    display.plot_ppi_map('ZDR_smooth', 0, title='ZDR (smooth)', cmap='pyart_HomeyerRainbow',
                         vmin=0, vmax=6, colorbar_label='', ax=ax, resolution='10m', projection=projection,
                         fig=fig)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-50, 50), ylim=(-50, 50), ax=ax)
    #display.set_limits(xlim=(-75, -55), ylim=(75, 95), ax=ax)
    #display.plot_point(PIPS_lon, PIPS_lat, 'r*', ms=20)
    display.ax.set_extent([40000, 90000, -30000, 20000], crs=projection)
    #ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'r*', ms=20)
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [ ]:
for radar_obj in radar_dict['radarsweeplist'][:1]:
    
    kdp, _, _ = pyart.retrieve.kdp_schneebeli(radar_obj, kdp_field='KDP', psidp_field='PHI', 
                                              band='S')
    
    radar_obj.add_field('KDP', kdp, replace_existing=True)
    
#     rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radar_obj, rhv_field='RHO', refl_field='REF',
#                                                               min_ncp=None, min_rhv=0.95,
#                                                               min_refl=5., max_refl=None) 
    #print(rhoHV_filter.gate_excluded)
#    radar_obj.fields['ZDR']['data'] = np.ma.masked_where(rhoHV_ref_filter.gate_excluded, radar_obj.fields['ZDR']['data'])
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    display = pyart.graph.RadarMapDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
#     projection = ccrs.LambertConformal(central_latitude=PIPS_lat,
#                                        central_longitude=PIPS_lon)
    # plot dBZ
    ax = fig.add_subplot(111)
    print("Time: ", radar_obj.metadata['start_datetime'])
    print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('REF', 0, title='dBZ', cmap='pyart_HomeyerRainbow',
#                          vmin=0., vmax=60, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    # plot D0
#     ax = fig.add_subplot(111)
#     print("Time: ", radar_obj.metadata['start_datetime'])
#     print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('D0', 0, title='D0', cmap='viridis',
#                          vmin=0., vmax=4., colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
                         # min_lon=-88., max_lon=-86., min_lat=40., max_lat=41.)
    # plot KDP
    
    display.plot_ppi_map('KDP', 0, title='KDP',
                         vmin=0, vmax=6, colorbar_label='', ax=ax, resolution='10m', projection=projection,
                         fig=fig)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-50, 50), ylim=(-50, 50), ax=ax)
    #display.set_limits(xlim=(-75, -55), ylim=(75, 95), ax=ax)
    #display.plot_point(PIPS_lon, PIPS_lat, 'r*', ms=20)
    display.ax.set_extent([40000, 90000, -30000, 20000], crs=projection)
    #ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'r*', ms=20)
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [ ]:
sounding_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/2016/soundings/NWSRAOB/'
sounding_name = 'BMX_043016_0Z.txt'
sounding_path = os.path.join(sounding_dir, sounding_name)
sounding = pd.read_csv(sounding_path,header=[1,2],delim_whitespace=True,skiprows=[3,4])
hght = np.array(sounding["HGHT"]).squeeze()
temp = np.array(sounding["TEMP"]).squeeze()

In [ ]:
for radar_obj in radar_dict['radarsweeplist'][:1]:
    h_dict, t_dict = pyart.retrieve.map_profile_to_gates(temp, hght, radar_obj)
    radar_obj.add_field('sounding_temperature', t_dict, replace_existing=True)
    radar_obj.add_field('height', h_dict, replace_existing=True)
    
    hydro_class = pyart.retrieve.hydroclass_semisupervised(radar_obj, refl_field='REF', zdr_field='ZDR', 
                                                           rhv_field='RHO', kdp_field='KDP', temp_field='sounding_temperature', 
                                                           hydro_field='radar_echo_classification')
    radar_obj.add_field('radar_echo_classification', hydro_class, replace_existing=True)
#     rhoHV_ref_filter = pyart.correct.moment_based_gate_filter(radar_obj, rhv_field='RHO', refl_field='REF',
#                                                               min_ncp=None, min_rhv=0.95,
#                                                               min_refl=5., max_refl=None) 
    #print(rhoHV_filter.gate_excluded)
#    radar_obj.fields['ZDR']['data'] = np.ma.masked_where(rhoHV_ref_filter.gate_excluded, radar_obj.fields['ZDR']['data'])
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    display = pyart.graph.RadarMapDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
#     projection = ccrs.LambertConformal(central_latitude=PIPS_lat,
#                                        central_longitude=PIPS_lon)
    # plot dBZ
    ax = fig.add_subplot(111)
    print("Time: ", radar_obj.metadata['start_datetime'])
    print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('REF', 0, title='dBZ', cmap='pyart_HomeyerRainbow',
#                          vmin=0., vmax=60, colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
    # plot D0
#     ax = fig.add_subplot(111)
#     print("Time: ", radar_obj.metadata['start_datetime'])
#     print("Elevation: ", radar_obj.elevation['data'][-1])
#     display.plot_ppi_map('D0', 0, title='D0', cmap='viridis',
#                          vmin=0., vmax=4., colorbar_label='', ax=ax, resolution='10m', projection=projection,
#                          fig=fig)
                         # min_lon=-88., max_lon=-86., min_lat=40., max_lat=41.)
    # plot ZDR

    display.plot_ppi_map('radar_echo_classification', 0, title='radar_echo_classification', 
                         ax=ax, resolution='10m', projection=projection,
                         fig=fig)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-50, 50), ylim=(-50, 50), ax=ax)
    #display.set_limits(xlim=(-75, -55), ylim=(75, 95), ax=ax)
    #display.plot_point(PIPS_lon, PIPS_lat, 'r*', ms=20)
    display.ax.set_extent([40000, 90000, -30000, 20000], crs=projection)
    #ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'r*', ms=20)
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()